# Download Udacity Self Driving Car Dataset

> From [Roboflow](https://public.roboflow.com/object-detection/self-driving-car)

> **Download the data:**

In [ ]:
import os

!mkdir UdacitySelfDrivingCarDataset
%cd UdacitySelfDrivingCarDataset

!curl -L "https://public.roboflow.com/ds/82j9gSZJ1E?key=SUuRj4rXGx" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
!mv /content/UdacitySelfDrivingCarDataset/export/images /content/UdacitySelfDrivingCarDataset/images
!mv /content/UdacitySelfDrivingCarDataset/export/labels /content/UdacitySelfDrivingCarDataset/labels
!rm -rf /content/UdacitySelfDrivingCarDataset/export

%cd ../

txt_dir = 'UdacitySelfDrivingCarDataset/labels'
img_dir = 'UdacitySelfDrivingCarDataset/images'
pre_counter_name = 'UdacitySelfDrivingCar'

for counter, image in enumerate(os.listdir(img_dir)):
    name = image.replace('.jpg', '')
    os.rename(f'{img_dir}/{name}.jpg', f'{img_dir}/{pre_counter_name}_{counter:05d}.jpg')
    os.rename(f'{txt_dir}/{name}.txt', f'{txt_dir}/{pre_counter_name}_{counter:05d}.txt')

> **Define the wanted classes:**


In [ ]:
classes = [
    'biker', 
    'car', 
    'pedestrian', 
    'trafficLight', 
    'trafficLight-Green', 
    'trafficLight-GreenLeft', 
    'trafficLight-Red', 
    'trafficLight-RedLeft', 
    'trafficLight-Yellow', 
    'trafficLight-YellowLeft', 
    'truck'
]

wanted_classes = ['car', 'pedestrian', 'truck']
wanted_ids = [i for i, cls in enumerate(classes) if cls in wanted_classes]
classes_id_map = {cls: id for cls, id in zip(wanted_classes, wanted_ids)}

new_classes_id_map ={
    'car': 5,
    'truck': 6,
    'pedestrian': 0
}

def id_updater(curr_id):
    if curr_id == 1:   # car
        return 5
    if curr_id == 10:  # truck
        return 6
    if curr_id == 2:   # pedestrian
        return 0
    else:
        return -1
    
print(f'\nWanted classes: {wanted_classes}   |   Wanted ids: {wanted_ids}\n')
print(f'Classes id map: {classes_id_map}\n')
print(f'New classes id map: {new_classes_id_map}\n')


Wanted classes: ['car', 'pedestrian', 'truck']   |   Wanted ids: [1, 2, 10]

Classes id map: {'car': 1, 'pedestrian': 2, 'truck': 10}

New classes id map: {'car': 5, 'truck': 6, 'pedestrian': 0}



> **Remove the unwanted classes and set the new classes id:**

In [ ]:
def load_txt_to_labels(txt_path):
    """loads a label from txt file"""
    txt_file = open(txt_path, "r")
    label = [i.split(' ') for i in txt_file.read().split('\n') if i]
    txt_file.close()
    return label

def labels_to_txt_str(labels):
    """convert nested list to string for txt output"""
    label_str = '\n'.join(
        [' '.join([str(cell) for cell in row]) for row in labels]
    )
    return label_str

def save_labels_to_txt(txt_path, label_str):
    """save new label to txt file"""
    txt_file = open(txt_path, "w")
    txt_file.write(label_str)
    txt_file.close()


from tqdm.autonotebook import tqdm

txt_files = sorted(os.listdir(txt_dir))
for txt_file in tqdm(txt_files):
    txt_path = f'{txt_dir}/{txt_file}'
    if txt_file[-3:] == 'txt':
        labels = load_txt_to_labels(txt_path)
        labels_new = []
        for label in labels:
            label[0] = id_updater(int(label[0]))
            if label[0] > 0:
                labels_new.append(label)

        save_labels_to_txt(txt_path, labels_to_txt_str(labels_new))

In [ ]:
!zip -r UdacitySelfDrivingCarDataset.zip UdacitySelfDrivingCarDataset

> **Copy to drive:**

In [ ]:
from google.colab import drive
%cp "/content/UdacitySelfDrivingCarDataset.zip" "/content/drive/MyDrive/Colab Notebooks/Technion 044167 - Object Tracking Project"
drive.flush_and_unmount()